# Class Syntax

- Notice that you need `this` to access members from other members.
- Members are **public by default** to match JavaScript where everything is public.
- Note that methods are like functions by **no function keyword**.
- For the most part, you have to declare variables in the body (unlike Python).
  - the one exception is in **parameter properties** where you can declare fields in a constructor by using an access keyword like _public_.
- Java-like inheritance syntax
- everything is **virtual**
- no `let` or `const` for fields
  - they all act like let by default and can be made to act const-ish with `readonly`


In [ ]:
// Worth noting: all members must be accessed with 'this' (similar to Python but without arg for it)

(() => {
  class Animal {
    // Public fields
    public name: string;
    age: number; // Public is the default to match JavaScript.

    // Private field
    private species: string;

    // Protected field
    protected habitat: string;

    // Static field
    static count: number = 0; // reset on each instance (not like Python)

    // Static method
    static getCount(): number {
      return Animal.count;
    }

    // Constant fields
    private readonly secret = "bla";
    private readonly notSet: string;

    constructor(name: string, age: number, species: string) {
      this.name = name;
      this.age = age;
      this.species = species;
      this.habitat = "Unknown";

      Animal.count++; // Increment count on each instance creation

      this.notSet = "set now"; // Can only set one time
    }

    // Getter method for species (not a property).
    public getSpecies(): string {
      return this.species;
    }

    // Setter method for habitat (not a real property).
    protected setHabitat(habitat: string): void {
      this.habitat = habitat;
    }

    introduce(): void {
      // public by default
      console.log(`Hi, I'm ${this.name} and I'm ${this.age} years old.`);
    }
  }

  class Dog extends Animal {
    // Static field
    static totalBarks: number = 0;

    // Instance property with getter
    get myBreed(): string {
      return this.breed;
    }

    // Instance property with getter and setter
    get home(): string {
      return this.habitat;
    }
    set home(home: string) {
      this.setHabitat(home);
    }

    // breed is a 'parameter property' (declared in constructor only)
    // this is often used for services and other injected things
    constructor(name: string, age: number, private breed: string) {
      super(name, age, "Dog");
      // Don't need to do anything with breed here (it's assigned already)
    }

    // Override method
    introduce(): void {
      console.log(
        `Woof! My name is ${this.name}, I'm ${this.age} years old, and I'm a ${this.breed}.`
      );
    }

    // Static method
    static getTotalBarks(): number {
      return Dog.totalBarks;
    }

    // New method
    bark(): void {
      Dog.totalBarks++;
      console.log("Woof! Woof!");
    }
  }

  const animal = new Animal("Max", 5, "Unknown");
  const dog = new Dog("Buddy", 3, "Golden Retriever");

  animal.introduce(); // Output: Hi, I'm Max and I'm 5 years old.
  dog.introduce(); // Output: Woof! My name is Buddy, I'm 3 years old, and I'm a Golden Retriever.
  dog.bark(); // Output: Woof! Woof!

  console.log(Animal.getCount()); // Output: 2
  console.log(Dog.getTotalBarks()); // Output: 1

  dog.home = "USA";
  console.log(dog.home); // Output: USA

  // Accessing private and protected properties
  //console.log(animal.species); // Error: Property 'species' is private
  //console.log(animal.habitat); // Error: Property 'habitat' is protected
})();


Hi, I'm Max and I'm 5 years old.
Woof! My name is Buddy, I'm 3 years old, and I'm a Golden Retriever.
Woof! Woof!
2
1
USA


undefined

# Nullable Members


In [4]:
(() => {
  class NullableMembers {
    optionalName?: string;
    nullableName: string | null = null;
    assertedName!: string;

    constructor() {
      // We're supposed to fill this by
      // end of constructor.
      this.assertedName = "something";
    }

    f() {
      console.log(this.optionalName);
      console.log(this.nullableName);
      console.log(this.assertedName);
    }
  }

  const n = new NullableMembers();
  n.f();
})();


undefined
null
something


undefined

# new


In [6]:
(() => {
  class MyClass {}

  const m = new MyClass(); // required
  // n = MyClass();  // Invalid
})();


undefined

# Default Constructor


In [9]:
(() => {
  class MyClass {
    // no constructor needed
  }

  const m = new MyClass();
})();


undefined

# Initializers Referencing Each Other


In [22]:
(() => {
  class MyClass {
    a = 5;
    b = this.a * this.a; // have 'this' even before c'tor
    c = this.f(); // works even though instance method
    // also note that this is not a constant expression

    f() {
      return 10;
    }
  }

  console.log(new MyClass().b);
  console.log(new MyClass().c);
})();


25
10


undefined

# super() call required!

Just like Python - the base won't be automatically called. However, unlike Python, the compiler will force you.


In [25]:
(() => {
  class MyBaseClass {
    constructor() {
      console.log("base called");
    }
  }

  class MyDerivedClass extends MyBaseClass {
    constructor() {
      console.log("do this first");
      super();
      console.log("derived called");
    }
  }

  new MyDerivedClass();
})();


do this first
base called
derived called


undefined

# Construction Order

It works like you'd expect from C++, mostly. Fields come before c'tor so that c'tor can refer to fields, and whole base comes before whole derived. However, one key stickikng point is that instead of an initialization list, you call the base constructor in the constructor body, and that introduces some weird rules:

- you can't use `this` until after `super()` is called.
- the base fields aren't created until `super()` starts.


In [29]:
(() => {
  class MyBaseClass {
    x = this.f();
    private c = 5;

    constructor() {
      console.log(this.c);
      console.log(this.x);

      console.log("base constructor");
    }

    f() {
      console.log("base fields");
      return 0;
    }
  }

  class MyDerivedClass extends MyBaseClass {
    private c2 = 5;
    y = this.g();

    constructor() {
      // These lines won't compile because they
      // access 'this' before super().
      //console.log(this.c2);
      //console.log(this.y);

      console.log("pre-super");
      super();
      console.log("post-super");
    }

    g() {
      console.log("derived fields");
      return 0;
    }
  }

  new MyDerivedClass();
})();


pre-super
base fields
5
0
base constructor
derived fields
post-super


undefined

# Inheritance and Field Names

Fields are virtualized just like methods.

In addition, note the somewhat strange limitation that you can't define a private member with the same name in a derived class. This is probably a side effect of the virtualization.


In [41]:
(() => {
  class MyBaseClass {
    private x = 10;
    protected y = 20;
    public z = 30;
  }

  class MyDerivedClass extends MyBaseClass {
    // private x = 100;  // ILLEGAL
    protected y = 200;
    public z = 300;
  }

  const m = new MyDerivedClass();
  console.log(m.z);
  const n: MyBaseClass = m;
  console.log(n.z);
})();


300
300


undefined

# Multiple Inheritance


**Not supported** directly by the language if you're talking about actual classes. You can inherit **1 class** directly (and others indirectly in a chain) and implement **unlimited interfaces**, like C# and Java.

If you want to do a **mixin** like you'd have in C++, you have to hack it in via the constructor (eg. adding keys to the object from another object). There may be libraries for this.


# Abstract Base Class


In [48]:
(() => {
  abstract class Animal {
    // If you try to inherit Animal in a
    // non-abstract class without implementing
    // this, the compilation will fail.
    abstract makeSound(): void;

    move(): void {
      console.log("Moving...");
    }
  }

  class Dog extends Animal {
    makeSound(): void {
      console.log("Woof!");
    }
  }

  class Cat extends Animal {
    makeSound(): void {
      console.log("Meow!");
    }
  }

  // const animal = new Animal();  // ILLEGAL

  const dog = new Dog();
  dog.makeSound(); // Output: Woof!
  dog.move(); // Output: Moving...

  const cat = new Cat();
  cat.makeSound(); // Output: Meow!
  cat.move(); // Output: Moving...
})();


Woof!
Moving...
Meow!
Moving...


undefined

# Interface

Note the use of **implements** instead of **extends**.


In [51]:
(() => {
  // Define an interface
  interface Person {
    name: string; // Even fields must be 'implemented' by subclass.
    age: number;
    greet(): void; // Non-abstract implementers must implement all methods.
  }

  // Implement the interface
  class Employee implements Person {
    name: string;
    age: number;

    constructor(name: string, age: number) {
      this.name = name;
      this.age = age;
    }

    greet(): void {
      console.log(
        `Hello, my name is ${this.name} and I'm ${this.age} years old.`
      );
    }
  }

  // Create an instance of the class
  const employee: Person = new Employee("John Doe", 30);
  employee.greet(); // Output: Hello, my name is John Doe and I'm 30 years old.
})();


Hello, my name is John Doe and I'm 30 years old.


undefined

# Multiple Interfaces

TODO: find out if possible to overload in Duck where the signatures of name are different in CanFly and CanSwim (but more importantly, hope that never comes up).


In [58]:
(() => {
  // Define interfaces
  interface CanFly {
    fly(): void;
    // Common name in both interfaces.
    name(): void;
  }

  interface CanSwim {
    swim(): void;
    // Common name in both interfaces.
    name(): void;
  }

  // Implement interfaces
  class Bird implements CanFly {
    fly(): void {
      console.log("Bird is flying.");
    }

    // Same implementation used for both interfaces.
    name(): void {
      console.log("Bird Name.");
    }
  }

  class Fish implements CanSwim {
    swim(): void {
      console.log("Fish is swimming.");
    }

    name(): void {
      console.log("Fish Name.");
    }
  }

  class Duck implements CanFly, CanSwim {
    fly(): void {
      console.log("Duck is flying.");
    }

    swim(): void {
      console.log("Duck is swimming.");
    }

    name(): void {
      console.log("Duck Name.");
    }
  }

  // Create instances
  const bird = new Bird();
  bird.fly(); // Output: Bird is flying.

  const fish = new Fish();
  fish.swim(); // Output: Fish is swimming.

  const duck = new Duck();
  duck.fly(); // Output: Duck is flying.
  duck.swim(); // Output: Duck is swimming.
  duck.name();
})();


Bird is flying.
Fish is swimming.
Duck is flying.
Duck is swimming.
Duck Name.


undefined

# Treating Class as Interface


In [62]:
(() => {
  class MyInterface {
    greet(): void {
      console.log("Hello!");
    }
  }

  // Throw away all implementations in MyInterface
  // and treat it as a pure interface.
  class MyClass implements MyInterface {
    greet(): void {
      console.log("Greetings!");
    }
  }

  const instance: MyInterface = new MyClass();
  instance.greet(); // Output: Greetings!
})();


Greetings!


undefined

# Private Interface Members

Once you have private members on a class, you cannot use it as a pure interface. You must extend it instead.

Interfaces cannot have private things.


In [14]:
(() => {
  class Person {
    name: string;
    private age: number;
  }

  // COMPILE ERROR
  class Employee implements Person {
    name: string;
    // It doesn't matter if you include this or not.
    //private age: number;
  }
})();


Error: Line 8, Character 9
  class Employee implements Person {
________^
TS2720: Class 'Employee' incorrectly implements class 'Person'. Did you mean to extend 'Person' and inherit its members as a subclass?
  Property 'age' is missing in type 'Employee' but required in type 'Person'.

# Nested/Private Classes

You are **not allowed** to define classes inside classes or make a class private at the file level. There are tricks you can do like **barrel** them into a module and only export the ones you want to be public.


In [65]:
(() => {
  // This should ideally be inside ParentClass, but it is
  // illegal to do so, no matter what access level you say.
  class HelperClass {
    private prefix: string;

    constructor() {
      this.prefix = "Processed: ";
    }

    public process(data: number[]): string[] {
      return data.map((item) => this.prefix + item.toString());
    }
  }

  class ParentClass {
    private data: number[];

    constructor() {
      this.data = [1, 2, 3];
    }

    public processData(): void {
      const helper = new HelperClass(); // Creating an instance of the private helper class
      const processedData = helper.process(this.data);
      console.log(processedData);
    }
  }

  const parent = new ParentClass();
  parent.processData(); // Output: ['Processed: 1', 'Processed: 2', 'Processed: 3']
})();


[ 'Processed: 1', 'Processed: 2', 'Processed: 3' ]


undefined

# Structural Typing

This is where it gets really **idiomatic**. Instead of having to explicitly inherit an interface, if an object structurally matches the interface, it passes the compiler.

This is like **duck typing** but more strict.

This is a case where **instanceof** can really lead to wrong results because when an object is duck typed, it doesn't pass the manual check. To do a runtime check on this, you'd have to do your own property checks.


In [81]:
(() => {
  interface Animal {
    name: string;
    makeSound: () => void;
  }

  // Doesn't inherit Animal, but sure looks like one.
  class Dog {
    name: string;
    makeSound(): void {
      console.log("Woof!");
    }
  }

  // Only knows about Animal, the interface.
  function animalSound(animal: Animal): void {
    animal.makeSound();
  }

  // Create a dog (and thus animal) without
  // calling the constructor becuase it looks
  // right.
  const dog: Dog = {
    name: "Buddy",
    // Basically treating like an interface.
    makeSound() {
      console.log("Quack!"); // duck typed implementation
    },
  };

  animalSound(dog); // Output: "Quack!"
  animalSound({
    name: "Buddy2",
    makeSound() {
      console.log("Quack2!");
    },
  });

  //console.log(dog instanceof Animal); // Doesn't even compile!
  console.log(dog instanceof Dog); // false because of duck typing
})();


Quack!
Quack2!
false


undefined

# Operator Overloading

**not supported**


# Object

_All classes_ inherit from Object even if no base class.


In [82]:
(() => {
  class MyClass {
    value: number;

    constructor(value: number) {
      this.value = value;
    }

    toString(): string {
      return `MyClass { value: ${this.value} }`;
    }

    valueOf(): number {
      return this.value;
    }

    equals(other: MyClass): boolean {
      // Note that this is friendly to structural typing.
      return this.value === other.value;
    }
  }

  const obj = new MyClass(42);
  console.log(obj.toString()); // Output: MyClass { value: 42 }

  const primitiveValue = obj.valueOf();
  console.log(primitiveValue); // Output: 42

  const otherObj = new MyClass(42);
  console.log(obj.equals(otherObj)); // Output: true
  // Reference equality still fails
  console.log(obj == otherObj); // Output: false
  console.log(obj === otherObj); // Output: false
})();


MyClass { value: 42 }
42
true
false
false


undefined

# Boxing

Boxing of primitive types to reference types is quite clunky compared to other languages.


In [111]:
(() => {
  const x = 5;

  // This line isn't even allowed because
  // number is a primitive that doesn't
  // inherit Object and can't be treated
  // like it's in the class hierarchy.

  //console.log(x instanceof Object);

  const s: string = "hi";

  // Even string is not an Object!
  //console.log(s instanceof Object);

  // Under the hood, these convert to a
  // boxed type that inherits object!
  console.log(x.toString());
  console.log(s.toString());

  // You can use the boxed types directly
  // like this.
  const x_b = Number(x);
  const s_b = String(s);

  // But you still can't treat them like
  // instances of Object (what?????!!!!).
  //console.log(x_b instanceof Object);
  //console.log(s_b instanceof Object);
})();


5
hi


undefined

# Adding Members to Objects and Functions


In [93]:
(() => {
  // Python style setting on instance doesn't work.
  class MyClass {
    x = 10;
  }

  const m = new MyClass();
  //m.y = 30; // ILLEGAL

  const n = {};
  //n.y = 30; // ILLEGAL

  // Python style setting on function works.
  // Probably to support decorators.
  const f = () => {};
  f.y = 30; // LEGAL
  f();
  console.log(f.y); // Output: 30

  // Dynamic object
  interface MyObject {
    [key: string]: any;

    //x: number;  // other members will become required in the {} on creation
  }
  // Even though created as a {}, using the MyObject interface
  const myObj: MyObject = {};

  myObj.field1 = "value1";
  myObj.field2 = 42;
  myObj.field3 = { foo: "bar" };

  console.log(myObj.field1); // Output: value1
  console.log(myObj.field2); // Output: 42
  console.log(myObj.field3); // Output: { foo: 'bar' }
})();


30
value1
42
{ foo: 'bar' }


undefined

# Object Indexer

Maps don't have the indexer syntax like in Python, but objects do!

**WARNING**: mixing of string names and syntax names for object members can cause problems with **minification**. The syntax access will get minified but the strings will not. So something that works in a dev build may not work in production.


In [95]:
(() => {
  const myObj = {
    field1: "value1",
    field2: 42,
    field3: { foo: "bar" },
  };

  const fieldName = "field1";
  console.log(myObj[fieldName]); // Output: value1

  const dynamicField = "field2";
  console.log(myObj[dynamicField]); // Output: 42
})();


value1
42


undefined

# Copying Objects


In [98]:
(() => {
  const myObj = {
    field1: "value1",
    field2: 42,
    field3: { foo: "bar" },
  };

  const myCopy = {
    ...myObj,
    field3: "replaced", // updates field copied in
    field4: "new", // brand new field
    // Note that if you did the ... here, field3 would
    // get set back to the myObj version.
  };

  myObj.field1 = "newvalue1"; // no effect on myCopy
  console.log(myCopy);
})();


{ field1: 'value1', field2: 42, field3: 'replaced', field4: 'new' }


undefined

# Ommitting Key Names

This works because {} is not used as a set like in other languages. So when the compiler sees {}, it already knows it's an object.


In [102]:
(() => {
  const field1 = 10;
  const field2 = 20;
  const field3 = 30;

  const myObj = {
    // If we already have a variable with the same
    // name, we can just say it once.
    field1,
    field2,
    // Or we can set it explicitly too.
    // If the name is different, we have to.
    field3: field3,
  };

  console.log(myObj);
})();


{ field1: 10, field2: 20, field3: 30 }


undefined

# Declaration Order

TypeScript is **pretty flexible** about order of declarations at the same level.

Also note that inner names can **hide outer** names as you'd expect in any language.

However, you **cannot redeclare the same name at the same level**, which is why all my jupyter cells have immediately called lambdas.


In [115]:
(() => {
  // We'll define f later.
  const x = f();

  function f() {
    return 5;
  }

  console.log(x);
})();


5


undefined

In [117]:
(() => {
  // Obviously this can't work.
  //const x = y;
  const y = 5;
})();


undefined

In [120]:
(() => {
  class A {
    f(): B {
      return new B();
    }
  }

  class B {}

  const a = new A();
  a.f();
})();


undefined

In [122]:
(() => {
  // This one doesn't work
  /*
  class A extends B {
  }
  
  class B {
  }
  
  const a = new A();
  */
})();


undefined

In [2]:
(() => {
  // This is even easier because when you say 'this' it already
  // knows where to look.
  class MyClass {
    x = this.f();

    g() {
      return this.f();
    }
    f() {
      return 5;
    }
  }
})();


undefined

# String Keys


When creating an object instance, you can provide strings intead of straight keys as the keys.

An important use case for this is to get around **minification**. Any symbol in code can be changed when the code is minified, so if you depended on a specific key existing by name, a minified version of the call may check the wrong key. An example of this is in **reactive forms** when you've specified the names of fields both in code and in the HTML template.

This goes together with using **ambient interfaces** (see Modules & Imports notebook) to declare interfaces that won't get minified when you need to pass them around in code.


In [6]:
(() => {
  interface MyInterface {
    x: number;
    y: number;
  }

  const m = {
    x: 10, // string as key
    y: 20, // normal key
  };
  console.log(m); // equivalent
})();


{ x: 10, y: 20 }


undefined

You can also do this on a class definition, but it's weird and probably not a good idea.


In [4]:
(() => {
  class MyClass {
    "a" = 5;
  }

  const m = new MyClass();
  console.log(m.a);
})();


5


undefined

# Partial Classes

** not supported **


# Partial<\T\>

This type is syntactic magic for turning an interface into the same interface with **all optional fields**.


In [9]:
(() => {
  interface MyInterface {
    x: number;
    y: number;
    z?: number;
  }

  // This would have an error if you didn't use Partial.
  const m: Partial<MyInterface> = {};
  m.x = 10;
  console.log(m);
})();


{ x: 10 }


undefined

# 'Override' keyword

This is a **newer** addition to TypeScript for explicitly specifying that you are **overriding a base method**.

By default, it is optional, but there is a compiler option (`noImplicitOverride`) to make it mandatory, which **Angular** turns on by default.


In [10]:
(() => {
  class MyBaseClass {
    f(): void {}
  }

  class MySubClass extends MyBaseClass {
    override f(): void {
      console.log("overridden!");
    }
  }

  const m = new MySubClass() as MyBaseClass;
  m.f();
})();


overridden!


undefined

In [16]:
(() => {
  class MyBaseClass {
    f(): void {}
  }

  class MySubClass implements MyBaseClass {
    // override keyword NOT ALLOWED if
    // interface instead of base class
    f(): void {
      console.log("overridden!");
    }
  }

  const m = new MySubClass() as MyBaseClass;
  m.f();
})();


overridden!


undefined